In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import undetected_chromedriver as uc
from selenium.webdriver.support import expected_conditions as EC
import time

# Create and open the CSV file for writing (not appending)
with open('411_2nd_1k_data_scrape_6.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    # Write the header row
    writer.writerow(['Phone Number', 'Name', 'Age', 'Location', 'Addresses', 'Landline 1', 'Landline 2'])
    
    # Initialize the WebDriver only once
    driver = uc.Chrome()
    
    # Open the input CSV file to read phone numbers
    with open('3nd 1k no file  - Sheet1.csv', 'r', newline='') as inputfile:
        reader = csv.reader(inputfile)
        next(reader)  # Skip the header row
        
        for row in reader:
            phone_number = row[0]
            
            try:
                driver.get('https://www.411.com/reverse-phone-lookup')
                
                # Find the input field and input the phone number
                phone_input = WebDriverWait(driver, 2).until(
                    EC.visibility_of_element_located((By.XPATH, "//input[@name='s_phone']"))
                )
                phone_input.clear()
                phone_input.send_keys(phone_number)
                
                # Click on the submit button
                submit_btn = WebDriverWait(driver, 2).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[@type='submit']"))
                )
                submit_btn.click()
                
                # Handle checkbox and TOS button (if present)
                try:
                    check_box = WebDriverWait(driver, 2).until(
                        EC.element_to_be_clickable((By.XPATH, "//input[@type='checkbox']"))
                    )
                    check_box.click()
                except Exception as e:
                    print(f"Checkbox error: {e}")
                
                try:
                    check_btn = WebDriverWait(driver, 2).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[@id='tos-button']"))
                    )
                    check_btn.click()
                except Exception as e:
                    print(f"TOS button error: {e}")
                
                # Wait for the main page to load
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@id='tos-overlay']")))
                
                # Extract information
                try:
                    name = driver.find_element(By.XPATH, "//*[@id='owner-name']/span[2]").text
                except Exception as e:
                    name = "Not Found"
                    print("Error retrieving name:", e)
                
                try:
                    age = driver.find_element(By.XPATH, "//*[@id='owner-age']/span[2]").text
                except Exception as e:
                    age = "Not Found"
                    print("Error retrieving age:", e)
                
                try:
                    location = driver.find_element(By.XPATH, "//*[@id='owner-location']/span[2]").text
                except Exception as e:
                    location = "Not Found"
                    print("Error retrieving location:", e)
                
                # Scroll the window
                driver.execute_script("window.scrollTo(0, 200);")
                
                try:
                    owner_addresses = driver.find_element(By.XPATH, "//div[@id='other-locations-0']").text
                except Exception as e:
                    owner_addresses = "Not Found"
                    print("Error retrieving addresses:", e)
                
                # Scroll further down
                driver.execute_script("window.scrollTo(300, 700);")
                time.sleep(2)
                
                try:
                    owner_landlines_number_1 = driver.find_element(By.XPATH, "//*[@id='landline-0']").text
                except Exception as e:
                    owner_landlines_number_1 = "Not Found"
                    print("Error retrieving landline 1:", e)
                
                try:
                    owner_landlines_number_2 = driver.find_element(By.XPATH, "//*[@id='landline-1']").text
                except Exception as e:
                    owner_landlines_number_2 = "Not Found"
                    print("Error retrieving landline 2:", e)
                
                # Write the extracted data to the CSV file
                writer.writerow([phone_number, name, age, location, owner_addresses, owner_landlines_number_1, owner_landlines_number_2])
            
            except Exception as e:
                print(f"An error occurred during processing: {e}")
            
        # Close the WebDriver
        driver.quit()
